## Downstream Task - speaker identification on 30 speakers

In [9]:
import os
import random
import numpy as np
import pandas as pd
from math import ceil
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import decomposition
from statistics import mode


In [63]:
speaker_id = ['1069','289','39','6181','8226','1334','3242','412','6272','831',
              '1553','332','5163','7402','8465','1737','3436','6000','7800',
              '8609','2159','3440','6064','7859','8770','2384','3807','6081',
              '8095','8975']

In [ ]:
DATA_FOLDER = r'/Users/ruoyuzhu/30-speakers/'

In [32]:
#train
import os

directory = DATA_FOLDER + 'train/quantized_3bit/'  # 1bit, 2bit, 3bit, 4bit, 8bit, embedding_6144

first = True
count = 0
X = np.zeros(6144)
y = np.zeros(1)


for entry in os.scandir(directory):
    if (entry.path.endswith(".npy") and entry.is_file()):
        
        #counting
        count += 1
        if (count%50 == 0):
            print('count',count/50,'path',entry.path)
            print(X.shape)
            
        if first:
            emb = np.load(entry.path) #load array 
            audio_size = emb.shape[0]  # how many 1s clips in each file
            true_label = entry.path.split('/')[6].split('-')[0] #replace 3 by path number
            print(audio_size, true_label)
            print(emb)
            X = emb
            y = np.repeat(true_label, audio_size)
            print(y.shape)
            print(X.shape)
            first = False
        else:
            emb = np.load(entry.path)
            audio_size = emb.shape[0]
            true_label = entry.path.split('/')[6].split('-')[0]  #label type = 'string'
            X = np.concatenate((X, emb))
            next_label = np.repeat(true_label, audio_size)
            y = np.concatenate((y, next_label))

        

121 3436
[[7 0 0 ... 3 3 3]
 [7 0 0 ... 3 3 3]
 [7 0 0 ... 3 3 3]
 ...
 [0 7 0 ... 3 3 3]
 [0 7 0 ... 3 3 3]
 [6 7 0 ... 3 3 3]]
(121,)
(121, 6144)
count 1.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/7859-102518-0014.npy
(6039, 6144)
count 2.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/8770-295462-0007.npy
(11490, 6144)
count 3.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/6000-55211-0048.npy
(18084, 6144)
count 4.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/8465-246940-0021.npy
(24208, 6144)
count 5.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/6000-55211-0049.npy
(30201, 6144)
count 6.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/39-121916-0013.npy
(36399, 6144)
count 7.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/831-130739-0026.npy
(42132, 6144)
count 8.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/6181-216552-0097.npy
(47849, 6144)
count 9.0 path /Users/ruoyuzhu/30-speakers/train/quantized_3bit/17

In [33]:
import pandas as pd 
pd.DataFrame(X).to_csv("X_quantized_3bit.csv")
pd.DataFrame(y).to_csv("y_quantized_3bit.csv")

In [34]:
clf_quantized_3bit = RandomForestClassifier(random_state=0)
clf_quantized_3bit.fit(X, y)
print('Fitting model..')

Fitting model..


In [46]:
#test per audio
directory = DATA_FOLDER + 'test/quantized_3bit/'

y_true = []
y_pred = []
first = True
#emb_test = np.zeros(0)
for entry in os.scandir(directory):
    if (entry.path.endswith(".npy") and entry.is_file()):
        emb_test = np.load(entry.path)
        #audio_size = emb.shape[0]
        
        #print(entry.path)
        #print(emb_test)
        
        true_label = entry.path.split('/')[6].split('-')[0]  #label type = 'string'
        predict_label = mode(clf_quantized_2bit.predict(emb_test))
        #print(y_true, y_pred)
        y_true.append(true_label)
        y_pred.append(predict_label)
            


In [47]:
# per audio result 
conf_matrix = confusion_matrix(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)

In [48]:
acc

1.0

In [56]:
#test per frame
directory = directory = DATA_FOLDER + 'test/quantized_3bit/'

y_true = []
y_pred = []
first = True
for entry in os.scandir(directory):
    if (entry.path.endswith(".npy") and entry.is_file()):
        emb_test = np.load(entry.path)

        predict_label = clf_quantized_3bit.predict(emb_test)
               
        true_label = entry.path.split('/')[6].split('-')[0]  #label type = 'string'
                
        
        audio_size = emb_test.shape[0]
        true_label_long = [true_label] * audio_size
         
        y_pred.append(predict_label)
        y_true.append(true_label_long)



In [57]:
import itertools
y_pred_1d = list(itertools.chain.from_iterable(y_pred))
y_true_1d = list(itertools.chain.from_iterable(y_true))

In [58]:
len(y_pred_1d)

59515

In [59]:
# per frame result 
conf_matrix = confusion_matrix(y_true_1d, y_pred_1d)
acc = accuracy_score(y_true_1d, y_pred_1d)

In [61]:
acc

0.8545912795093674

In [ ]:
# save the model to disk
import pickle
filename = 'quantized_3bit_model.sav'
pickle.dump(clf_quantized_1bit, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(emb_test) # '6064'
print(result)